In [ ]:
import tensorflow as tf
import numpy as np
from IPython import display
from pathlib import Path
import png
import datetime
import math
import random

In [ ]:
r = png.Reader('lena-gr.png')
(width, height, rows_gen, info) = r.read()
print(width, height, info)
rows = [row for row in rows_gen]


In [ ]:

dropout = tf.keras.layers.Dropout(0.03)

model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=(2,)),
  tf.keras.layers.Flatten(input_shape=(2*width + 2*height - 2,)),
  tf.keras.layers.Dense(500, activation='leaky_relu'),
  tf.keras.layers.Dense(200, activation='leaky_relu'),
  tf.keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer='Adam',
              loss=tf.keras.losses.MeanSquaredError(),
              #metrics=['binary_accuracy', tf.keras.metrics.FalseNegatives(), tf.keras.metrics.FalsePositives()]
              )

tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_dtype=True, show_layer_names=True, expand_nested=True, show_layer_activations=True, show_trainable=True)
model.summary()
display.Image(f'model.png')

In [ ]:

class MySequence(tf.keras.utils.Sequence):

    def __init__(self, batch_size, randomize):
        self.batch_size = batch_size
        self.x_dim = []
        for x in range(width):
            xx = x / width * math.pi
            vect = [xx]
            for i in range(1, width):
                vect.append(math.sin(xx * i))
                vect.append(math.cos(xx * i))
            self.x_dim.append(np.array(vect, np.float32))
        self.y_dim = []
        for y in range(height):
            yy = y / height * math.pi
            vect = [yy]
            for i in range(1, height):
                vect.append(math.sin(yy * i))
                vect.append(math.cos(yy * i))
            self.y_dim.append(np.array(vect, np.float32))
        self.map = list(range(width * height))
        self.randomize = randomize
        if randomize:
            random.shuffle(self.map)

    def on_epoch_end(self):
        if self.randomize:
            random.shuffle(self.map)

    def __len__(self):
        return (width * height + self.batch_size - 1) // self.batch_size

    def __getitem__(self, index):
        pixels = range(index * self.batch_size, min(width * height, (index + 1) * self.batch_size))
        pixels = list(map(lambda i: self.map[i], pixels))
        return (
                np.array([np.concatenate((self.x_dim[i % width], self.y_dim[i // width]), axis=0) for i in pixels], np.float32),
                #np.array([[i % width / width * math.pi, i // width / height * math.pi] for i in pixels], np.float32),
                np.array([rows[i // width][i % width] * 0.003125 + 0.1 for i in pixels], np.float32)
               )

In [ ]:
train_data = MySequence(1000, True)

#model.optimizer.learning_rate = 0.001
#dropout.rate = 0.99

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=train_data,
          epochs=30,
          callbacks=[tensorboard_callback]
          )


In [ ]:

#all_pixels = range(width * height)
#inputs = np.array([[i % width / width, i // width / height] for i in all_pixels], np.float32)
#outputs = model.predict(inputs)

train_data = MySequence(14000, False)

outputs = []
for i in range(len(train_data)):
    x_data, _ = train_data[i]
    r = model.predict(x_data)
    outputs.extend(r)

outputs = np.concatenate(outputs, axis=0)

w = png.Writer(width, height, greyscale=True, bitdepth=8)
with open('out.png', 'wb') as fd:
    w.write(fd, (((outputs[i:i+width] - 0.01) * 320).astype(np.uint8) for i in range(0, width * height, width)))

display.Image(f'out.png')


In [ ]:
display.Image(f'lena-gr.png')